In [ ]:
# %matplotlib inlineb
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

2021.07.06

计算水汽通量的积分值，在等压面上计算并积分

有冗长的整理部分，通过dict选取变量，并且整理到相应的am，jja上，再进行计算，绘制平均

# 数据读取

In [ ]:
ds_wind = {}
ds_qv   = {}

# diag数据包含9-1，需要去掉尾部
dir_in = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge/ke_daily_vi/"
ds_wind['vr'] = xr.open_mfdataset(dir_in + "*_VR_ke_daily_vi.nc")
ds_wind['vr'] = ds_wind['vr'].sel(Time = ds_wind['vr'].Time.dt.month.isin([4,5,6,7,8]))
dir_in = "/raid52/yycheng/MPAS/VR_postprocess/VR_merge/hum_theta_daily_vi/"
ds_qv['vr'] = xr.open_mfdataset(dir_in + "*_VR_hum_theta_daily_vi.nc")
ds_qv['vr'] = ds_qv['vr'].sel(Time = ds_qv['vr'].Time.dt.month.isin([4,5,6,7,8]))

dir_in = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/ke_daily_vi/"
ds_wind['rcm'] = xr.open_mfdataset(dir_in + "*_RCM_ke_daily_vi.nc")
ds_wind['rcm'] = ds_wind['rcm'].sel(Time = ds_wind['rcm'].Time.dt.month.isin([4,5,6,7,8]))
dir_in = "/raid52/yycheng/MPAS/RCM_postprocess/RCM_merge/hum_theta_daily_vi/"
ds_qv['rcm'] = xr.open_mfdataset(dir_in + "*_RCM_hum_theta_daily_vi.nc")
ds_qv['rcm'] = ds_qv['rcm'].sel(Time = ds_qv['rcm'].Time.dt.month.isin([4,5,6,7,8]))

In [ ]:
# 添加ERA5I观测数据
ds_wind['obs'] = {}

# 切换 ERA5I 坐标到MPAS输出结果，方便转换

rename_dict = {"level":"plevels", "lon":"longitude", "lat":"latitude", "time":"Time"}
# ds_wind['obs']['uReconstructZonal'].assign_coords(Time = Time)
# show converting coords
for rename_i in rename_dict:
    print(rename_i + " -----converting to----- " + rename_dict[rename_i])
# 读取数据后就整理坐标，和RCM的坐标一致，之后再进行计算
dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/u_98-17_daily/"
ds_wind['obs']["uReconstructZonal"] = xr.open_dataset(dir_in + "merge_u_daily.nc")\
    .rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)['uwnd']

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/v_98-17_daily/"
ds_wind['obs']["uReconstructMeridional"] = xr.open_dataset(dir_in + "merge_v_daily.nc")\
    .rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)['vwnd']

dir_in = "/raid52/yycheng/MPAS/REFERENCE/ERA5I_NC/ERA5I_NC_daily/shum_98-17_daily/"
ds_qv['obs'] = xr.open_dataset(dir_in + "merge_shum_daily.nc").rename(rename_dict).assign_coords(ds_qv['rcm']['qv'].coords)
ds_qv['obs'] = ds_qv['obs'].rename({"shum":"qv"})

In [ ]:
# ----- select data range -----
lat_sel     = (ds_qv['vr'].latitude >= 10) & (ds_qv['vr'].latitude <= 60)
lon_sel     = (ds_qv['vr'].longitude >= 70) & (ds_qv['vr'].longitude <= 140)
plevels_sel = (ds_qv['vr'].plevels >= 500) & (ds_qv['vr'].plevels <= 925)

# plevels_sel = (ds_qv['vr'].plevels == 100)
time_year    = (ds_qv['vr'].Time.dt.year >= 1998) # 时次相对较长，一开始使用1998年一年进行尝试
time_sel_am     = ds_qv['vr'].Time.dt.month.isin([4,5])
time_sel_jja    = ds_qv['vr'].Time.dt.month.isin([6,7,8])

sel_dict = {}
sel_dict['am']    = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":(time_sel_am & time_year)}
sel_dict['jja']   = {'longitude':lon_sel, "latitude":lat_sel, "plevels":plevels_sel, "Time":(time_sel_jja & time_year)}

In [ ]:
# ----- 选取变量 -----
mod_list    = ['obs', 'vr', 'rcm']
season_list = ['am', 'jja']
u_sel = {}
# u_sel['vr'] = {}
# u_sel['rcm'] = {}
# u_sel['vr']['am']  = ds_wind['vr']['uReconstructZonal'].isel(sel_dict_am)        # 读入选取数据到 vr_wind_sel
# u_sel['vr']['jja'] = ds_wind['vr']['uReconstructZonal'].isel(sel_dict_jja)        # 读入选取数据到 vr_wind_sel

v_sel = {}
# v_sel['vr'] = {}
# v_sel['rcm'] = {}
# v_sel['vr']['am']  = ds_wind['vr']['uReconstructMeridional'].isel(sel_dict_am)        # 读入选取数据到 vr_wind_sel
# v_sel['vr']['jja'] = ds_wind['vr']['uReconstructMeridional'].isel(sel_dict_jja)        # 读入选取数据到 vr_wind_sel

qv_sel = {}
# qv_sel['vr'] = {}
# qv_sel['rcm'] = {}
# qv_sel['vr']['am']  = ds_qv['vr']['qv'].isel(sel_dict_am)
# qv_sel['vr']['jja'] = ds_qv['vr']['qv'].isel(sel_dict_jja)

for imod in mod_list:
    # print(imod)
    u_sel[imod]  = {}
    v_sel[imod]  = {}
    qv_sel[imod] = {}
    for iseason in season_list:
        # print(iseason)
        u_sel[imod][iseason]  = ds_wind[imod]['uReconstructZonal'].isel(sel_dict[iseason])
        v_sel[imod][iseason]  = ds_wind[imod]['uReconstructMeridional'].isel(sel_dict[iseason])
        qv_sel[imod][iseason] = ds_qv[imod]['qv'].isel(sel_dict[iseason])

# 计算垂直积分

保存到变量 uflux_vi 和 vflux_vi 之中

In [ ]:
# count flux
uflux = {}
vflux = {}
# count flux vertical integration
uflux_vi = {}
vflux_vi = {}
sel_plevels_val = u_sel['vr']['am'].plevels

for imod in mod_list:
    print(imod)
    uflux[imod] = {}
    vflux[imod] = {}
    uflux_vi[imod] = {}
    vflux_vi[imod] = {}
    for iseason in season_list:
        print(iseason)
        uflux[imod][iseason] = u_sel[imod][iseason] * qv_sel[imod][iseason]
        vflux[imod][iseason] = v_sel[imod][iseason] * qv_sel[imod][iseason]

        # find plevels axis location
        plevels_axis = list(uflux[imod][iseason].dims).index('plevels')
        uflux_vi[imod][iseason] = np.trapz(uflux[imod][iseason], sel_plevels_val, axis = plevels_axis)
        vflux_vi[imod][iseason] = np.trapz(vflux[imod][iseason], sel_plevels_val, axis = plevels_axis)
# 

In [ ]:
# 计算绘图的平均值
uflux_vi_mean = {} # 平均u方向水汽通量
vflux_vi_mean = {} # 平均v方向水汽通量
absflux_vi_mean = {} # 水汽通量大小，平均图
for imod in mod_list:
    uflux_vi_mean[imod] = {}
    vflux_vi_mean[imod] = {}
    absflux_vi_mean[imod] = {}
    for iseason in season_list:
        uflux_vi_mean[imod][iseason] = uflux_vi[imod][iseason].mean(axis=0)
        vflux_vi_mean[imod][iseason] = vflux_vi[imod][iseason].mean(axis=0)
        absflux_vi_mean[imod][iseason] = np.sqrt(uflux_vi_mean[imod][iseason] ** 2 + vflux_vi_mean[imod][iseason] **2) 
# flux 等值线绘制大小

In [ ]:
# 整理绘图所使用的坐标变量
lat  = u_sel['vr']['jja'].latitude
lon  = u_sel['vr']['jja'].longitude
time = u_sel['vr']['jja'].Time
time_am = u_sel['vr']['am'].Time
time_jja = u_sel['vr']['jja'].Time
# vr_uflux_sum_addcoords = xr.DataArray(vr_uflux_sum, coords=[time,lat,lon], name = "qv uflux").mean('Time')
# vr_vflux_sum_addcoords = xr.DataArray(vr_vflux_sum, coords=[time,lat,lon], name = "qv vflux").mean('Time')
# vr_flux_all = np.sqrt(vr_uflux_sum_addcoords ** 2 + vr_vflux_sum_addcoords ** 2 )

# 临时文件

进行文件的输出部分 计算时间太长 需要一个半小时

进行flux垂直积分之后结果的输出，放置到 /raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/dyn/shum_flux 之中

合并了RCM的边界上的空缺，但是文件输出都是手动进行的，注意对变量、时次进行调整

In [ ]:
# 处理RCM 中边界的NAN的问题，合并成同样的fillvalues，之后赋值为 -1e30
for iseason in season_list:
    rcm_nan_index = (uflux_vi['rcm'][iseason][:,:,:] > 1e10)
    uflux_vi['rcm'][iseason][rcm_nan_index] = -1e30
    vflux_vi['rcm'][iseason][rcm_nan_index] = -1e30

In [ ]:
# 临时存放变量
# uflux_vi['obs']['am'].shape
iseason = 'jja'
uflux_vi_tonc = {}
vflux_vi_tonc = {}
encoding = {}
for imod in mod_list:
    # uflux_vi_tonc[imod] = xr.DataArray(uflux_vi[imod][iseason], coords=[time_jja,lat,lon], name=imod)
    vflux_vi_tonc[imod] = xr.DataArray(vflux_vi[imod][iseason], coords=[time_jja,lat,lon], name=imod)
    # 添加NAN时需要
    encoding[imod] = {"_FillValue":-1e30}

ds_out = xr.merge(list( vflux_vi_tonc.values() ))
ds_out.fillna(-1e30)

In [ ]:
dir_out = "/raid52/yycheng/MPAS/REFERENCE/TEMP_DATA/dyn/shum_flux/"
ds_out.to_netcdf("shum_vflux_"+iseason+".nc", encoding = encoding)
ds_out.close()

# 绘图部分

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot

# cmap = "coolwarm"
cmap = "gist_ncar"
# ----- get filter vars coords-----

cticks = np.arange(0,48,2)

#----- create plot -----
fig, axs = plot.subplots(ncols=3,nrows=2, proj=('cyl'))

#-----vr-----
axs[0].format(title='ERA5I AM')
m_contour= axs[0].contourf(lon, lat, absflux_vi_mean['obs']['am'], levels = cticks, cmap=cmap)
m_quiver = axs[0].quiver(lon[::10], lat[::10], uflux_vi_mean['obs']['am'][::10,::10], vflux_vi_mean['obs']['am'][::10,::10],scale = 400.)

axs[1].format(title='VR AM')
m_contour= axs[1].contourf(lon, lat, absflux_vi_mean['vr']['am'], levels = cticks, cmap=cmap)
m_quiver = axs[1].quiver(lon[::10], lat[::10], uflux_vi_mean['vr']['am'][::10,::10], vflux_vi_mean['vr']['am'][::10,::10],scale = 400.)
#-----rcm -----
axs[2].format(title='RCM AM')
m_contour= axs[2].contourf(lon, lat, absflux_vi_mean['rcm']['am'], levels = cticks, cmap=cmap)
m_quiver = axs[2].quiver(lon[::10], lat[::10], uflux_vi_mean['rcm']['am'][::10,::10], vflux_vi_mean['rcm']['am'][::10,::10],scale = 400.)
axs[2].quiverkey(m_quiver, 0.5, 1.2, label="20 (kg*m*hPa)/(s*kg)", U=20, labelpos="E")

axs[3].format(title='ERA5I JJA')
m_contour= axs[3].contourf(lon, lat, absflux_vi_mean['obs']['jja'], levels = cticks, cmap=cmap)
m_quiver = axs[3].quiver(lon[::10], lat[::10], uflux_vi_mean['obs']['jja'][::10,::10], vflux_vi_mean['obs']['am'][::10,::10],scale = 400.)

axs[4].format(title='VR JJA')
m_contour= axs[4].contourf(lon, lat, absflux_vi_mean['vr']['jja'], levels = cticks, cmap=cmap)
m_quiver = axs[4].quiver(lon[::10], lat[::10], uflux_vi_mean['vr']['jja'][::10,::10], vflux_vi_mean['vr']['am'][::10,::10],scale = 400.)

axs[5].format(title='RCM JJA')
m_contour= axs[5].contourf(lon, lat, absflux_vi_mean['rcm']['jja'], levels = cticks, cmap=cmap)
m_quiver = axs[5].quiver(lon[::10], lat[::10], uflux_vi_mean['rcm']['jja'][::10,::10], vflux_vi_mean['rcm']['am'][::10,::10],scale = 400.)


fig.colorbar(m_contour, cmap=cmap,loc='b', width=0.1,ticklabelsize=5,ticks=cticks,label="units : (kg*m*hPa)/(s*kg)")
# format作为整个proplot集中对属性进行修改的method
axs.format(
abc=True,
lonlim=(70, 140), latlim=(15, 55), coast=True,
#-----GEO axis-----
borders = True,
labels = True,
gridlabelsize = 5,
gridminor = False,

longrid  = True,
latgrid  = True,
# latlines = 10,
# lonlines = 10,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,65,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
#-----line label-----
# linewidth = 0.5,
suptitle="925-500hPa vertical specific humidity flux integration",
)
# #----- add contour tick labels -----
# # axs.clabel(m, inline=True, fontsize=5, colors = 'k', fmt='%4.0f',use_clabeltext=True, inline_spacing=0)
# fig.colorbar(m_contour, loc='b', cmap=cmap, width=0.1)

fig.patch.set_facecolor('white')
plt.savefig('./output_pic/water_flux_vi_98-17.png', dpi=600, facecolor= "white")